In [ ]:
"""
TODO:
1. add previous n_bookings as a feature
2. add number rooms booked previously as average
"""

In [ ]:
import pandas as pd
import lightgbm
import category_encoders as ce
from sklearn.preprocessing import CategoricalEncoder,OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from bayes_opt import BayesianOptimization
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np
import datetime

In [ ]:
train = pd.read_csv("train_5CLrC8b/train.csv")
test = pd.read_csv("test.csv")
sample_submission = pd.read_csv("sample_submission.csv")

In [ ]:
concated = pd.concat([train,test],ignore_index=True)

In [ ]:
concated['booking_date'] = pd.to_datetime(concated['booking_date'],dayfirst=True)
concated['checkin_date'] = pd.to_datetime(concated['checkin_date'],dayfirst=True)
concated['checkout_date'] = pd.to_datetime(concated['checkout_date'],dayfirst=True)

In [ ]:
concated['booking_month'] = concated['booking_date'].apply(lambda x:x.month)
concated['booking_year'] = concated['booking_date'].apply(lambda x:x.year)
concated['booking_day'] = concated['booking_date'].apply(lambda x:x.day)

concated['checkin_month'] = concated['checkin_date'].apply(lambda x:x.month)
concated['checkin_year'] = concated['checkin_date'].apply(lambda x:x.year)
concated['checkin_day'] = concated['checkin_date'].apply(lambda x:x.day)

concated['checkout_month'] = concated['checkout_date'].apply(lambda x:x.month)
concated['checkout_year'] = concated['checkout_date'].apply(lambda x:x.year)
concated['checkout_day'] = concated['checkout_date'].apply(lambda x:x.day)

In [ ]:
concated['booking_dayofweek'] = concated['booking_date'].apply(lambda x:x.dayofweek)

concated['checkin_dayofweek'] = concated['checkin_date'].apply(lambda x:x.dayofweek)

concated['checkout_dayofweek'] = concated['checkout_date'].apply(lambda x:x.dayofweek)

In [ ]:
concated['season_holidayed_code_is_na'] = concated.season_holidayed_code.apply(
    lambda x: 1 if pd.isna(x) else 0)
concated['state_code_residence_is_na'] = concated.state_code_residence.apply(
    lambda x: 1 if pd.isna(x) else 0)
concated['season_holidayed_code'] = concated.season_holidayed_code.apply(
    lambda x: 0 if pd.isna(x) else x)
concated['state_code_residence'] = concated.state_code_residence.apply(
    lambda x: 0 if pd.isna(x) else x)

In [ ]:
concated['actual_days_booked'] = concated.apply(
    lambda x: (x['checkout_date'] - x['checkin_date']).days, axis=1)

In [ ]:
concated['n_rooms_booked'] = concated.apply(
    lambda x: x['roomnights'] / x['actual_days_booked'], axis=1)

In [ ]:
concated['person_per_room'] = concated.apply(
    lambda x: (x['numberofadults'] + x['numberofchildren']) / x['n_rooms_booked'] if x['n_rooms_booked'] else 0,
    axis=1)

In [ ]:
def count_weekend(row):
    checkin_date = row['checkin_date']
    actual_days_booked = row['actual_days_booked']
    stayed_in_weekend = 0
    for i in range(actual_days_booked):
        new_day = (checkin_date + datetime.timedelta(days=i)).day_name()
        if new_day == "Saturday" or new_day == "Friday" or new_day == "Sunday":
            stayed_in_weekend += 1
    return stayed_in_weekend

In [ ]:
concated['n_weekend_days'] = concated.apply(count_weekend,axis=1)

In [ ]:
concated.drop(['reservation_id','booking_date','checkin_date','checkout_date','memberid'],axis=1,inplace=True)

In [ ]:

l=['member_age_buckets','cluster_code','reservationstatusid_code','resort_id']

encoder = ce.polynomial.PolynomialEncoder(l)
concated = encoder.fit_transform(concated)

#encoder = OrdinalEncoder(dtype=np.int32,)
#concated[['memberh_age_buckets','cluster_code','reservationstatusid_code','resort_id']] = encoder.fit_transform(
#    concated[['member_age_buckets','cluster_code','reservationstatusid_code','resort_id']]
#)

In [ ]:
# concated.to_csv("./concat_train_test_feat_engg.csv",index=False)

In [ ]:
concated.drop(['intercept'],axis=1,inplace=True)
concated.head()

In [ ]:
train = concated[:len(train)]
test = concated[-len(test):]

In [ ]:
all_predictions = []
all_mse = []
X = train.drop(['amount_spent_per_room_night_scaled'], axis=1)
Y = train['amount_spent_per_room_night_scaled']
for i in range(10):
    print(i)
    trainX, testX, trainY, testY = train_test_split(X, Y,test_size=0.15)
    lgb_model = lightgbm.LGBMRegressor(n_estimators=1000)
    lgb_model.fit(
        trainX,
        trainY,
        eval_set=(testX, testY),
        eval_metric='rmse',
        early_stopping_rounds=100,verbose = 0 
    )
    predictions = lgb_model.predict(test.drop(['amount_spent_per_room_night_scaled'], axis=1))
    test_pred = lgb_model.predict(testX)
    from sklearn.metrics import mean_squared_error
    from math import sqrt
    all_mse.append(sqrt(mean_squared_error(testY, test_pred)))
    all_predictions.append(predictions)

In [ ]:
#internal_pred = np.stack(internal_test_pred,axis=1).mean(axis=1)
final_predictions = np.stack(all_predictions,axis=1).mean(axis=1)

In [ ]:
sample_submission['amount_spent_per_room_night_scaled'] = final_predictions
sample_submission.to_csv(
    "poly_enc_1.csv",
    index=False)

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt
sqrt(mean_squared_error(y_test_internal, pred_orig_test))


In [ ]:
np.array(all_mse).mean(axis=0)

In [ ]:
all_mse